# Data Cleaning & Exploratory Data Analysis (EDA)

## Project: Customer Churn Prediction & Business Insights Dashboard

**Objective:**  
In this notebook, we begin our end-to-end data science project by exploring and cleaning the Telco Customer Churn dataset. The goal is to understand the structure of the data, identify key features influencing churn, and prepare the dataset for machine learning.

---

## Tasks Covered:
- Load and inspect raw customer data
- Clean and preprocess missing or inconsistent values
- Explore feature distributions and relationships with churn
- Visualize churn trends and key business insights
- Export a clean dataset for modeling 

---

In [ ]:
# Import libraries
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.style.use('ggplot')
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12,8)

--- 

Let’s load the raw CSV file into a DataFrame. We’ll display the first few rows and get an overview of column types and summary statistics to spot any issues.

In [ ]:
# Load and Inspect Data
df = pd.read_csv('../dataset/WA_Fn-UseC_-Telco-Customer-Churn.csv')
display(df.head(3))
df.info()
df.describe(include="all") 

---

The summary will tell us if any columns have missing values or incorrect types. We’ll  check for any “object” types in numeric-looking columns and count nulls.

In [ ]:
# Check for missing values and data types

missing_counts = df.isnull().sum()
print('Missing values per column:\n', missing_counts)

object_cols = df.select_dtypes(include='object').columns.tolist()
print('Object-typed columns:', object_cols)

--- 

Since the column 'TotalCharges' is object-typed and likely contains blanks, we will convert it to a numeric type, coerce errors to NaN, then drop any resulting missing rows


In [ ]:
# Convert TotalCharges to numeric and drop rows with missing data
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
print("Missing TotalCharges after conversion:", df['TotalCharges'].isnull().sum())

# Drop rows with any missing values
df.dropna(inplace=True)
print("Shape after dropping missing rows:", df.shape)


---

We will remove or transform columns that won't help with training the model. Specifically, 'customerID' since it is just an identifier, and 'SeniorCitizen' is numeric but will be better represented as categorical

In [ ]:
# Tidy up irrelevant or mis-typed columns 
df.drop(columns=['customerID'], inplace=True)

df['SeniorCitizen'] = df['SeniorCitizen'].map({0: 'No', 1: 'Yes'})

df.columns = df.columns.str.strip()

df.info()


---
### Churn Distribution

Before we go deeper, we will examine our target variable's distribution to see if we face any class imbalance issues

In [ ]:
# Visualize churn distribution
ax = sns.countplot(data=df, x='Churn')
ax.set_title("Overall Churn Distribution")
for p in ax.patches:
    ax.annotate(f"{p.get_height()}", 
                (p.get_x() + p.get_width() / 2, p.get_height()), 
                ha='center', va='bottom')
plt.ylabel("Count")
plt.show()


In [ ]:
churn_pct = df['Churn'].value_counts(normalize=True)*100
print(churn_pct.round(2))

- **26.5%** of customers have churned (`Churn = Yes`) and **73.5%** have stayed (`Churn = No`).
- This moderate class imbalance means we should use **stratified splits** or **class weights/resampling** when training models.

--- 
### Numeric Feature Histograms

We will now analyze the numerical features ('tenure', 'MonthlyCharges', 'TotalCharges') to see how their distributions differ between customers who left and those who stayed

In [ ]:

num_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
for col in num_cols:
    plt.figure()
    sns.histplot(data=df, x=col, hue='Churn', kde=True, bins=30)
    plt.title(f"{col} Distribution by Churn")
    plt.show()

In [ ]:
# Median tenure by churn
med_tenure = df.groupby('Churn')['tenure'].median()

# Mean monthly and total charges by churn
mean_month = df.groupby('Churn')['MonthlyCharges'].mean()
mean_total = df.groupby('Churn')['TotalCharges'].mean()

print("Median Tenure:\n", med_tenure.round(1))
print("\nMean Monthly Charges:\n", mean_month.round(2))
print("\nMean Total Charges:\n", mean_total.round(2))


- **Tenure**:  
  - Median tenure of churners is **10 months** vs. **38 months** for non‑churners.  
  - Over **50%** of churners leave within the first 10 months.

- **MonthlyCharges**:  
  - Churners pay on average **$74.44** vs. **$61.31** for retained customers.  
  - Higher bills correlate strongly with churn risk.

- **TotalCharges**:  
  - Churners’ average total spend is **$1,532** vs. **$2,555** for non‑churners.  
  - Distribution is right‑skewed; consider a log transform if used as a raw feature.


--- 
### Categorical Feature Countplots

We'll now look at how each categorical feature relates to churn. We’ll create count plots for each to spot patterns (e.g., contract type or tech support).

In [ ]:
# Analysis of categorical features
cat_cols = [c for c in df.select_dtypes('object').columns if c != 'Churn']
for col in cat_cols:
    plt.figure(figsize=(10,4))
    sns.countplot(data=df, x=col, hue='Churn')
    plt.title(f"{col} vs. Churn")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
# Function to print churn % per group
def churn_rate(col):
    rates = df.groupby(col)['Churn'].value_counts(normalize=True).mul(100).unstack()
    print(f"\nChurn rates by {col}:")
    display(rates.round(1))

for col in ['Contract','TechSupport','InternetService','PaymentMethod','SeniorCitizen']:
    churn_rate(col)

#### Contract
- **Month‑to‑month:** 42.7% churn  
- **One‑year:** 11.3% churn  
- **Two‑year:** 2.8% churn  
> **Business wise:** Offer early incentives to move month‑to‑month customers into annual contracts.

#### TechSupport
- **No Tech Support:** 41.6% churn  
- **Yes Tech Support:** 15.2% churn  
> **Opportunity:** Bundle or upsell support services for new customers.

#### InternetService
- **DSL:** 19% churn  
- **Fiber optic:** 41.9% churn  
- **No internet service:** 7.4% churn  
> **Insight:** Fiber‑optic users are most at risk—consider service‑quality guarantees or loyalty perks.

#### PaymentMethod
- **Electronic check:** 45.3% churn  
- **Mailed check:** 19.2% churn  
- **Bank transfer (automatic):** 16.7% churn  
- **Credit card (automatic):** 15.3% churn  
> **Action:** Encourage auto-pay enrollment to reduce churn.

#### SeniorCitizen
- **No (under 65):** 23.7% churn  
- **Yes (65+):** 41.7% churn  
> **Strategy:** Develop targeted engagement and educational outreach for senior customers.


--- 

### Correlation heatmap

Finally, we want an overall correlation view. We’ll one‑hot encode the DataFrame temporarily to compute pairwise correlations, including with a numeric churn flag.

In [ ]:
# Correlation heatmap including churn
tmp = pd.get_dummies(df.drop('Churn', axis=1), drop_first=True)
tmp['ChurnFlag'] = df['Churn'].map({'No': 0, 'Yes': 1})

plt.figure(figsize=(14,10))
sns.heatmap(tmp.corr(), cmap='coolwarm', center=0)
plt.title("Feature Correlations (including ChurnFlag)")
plt.show()

In [ ]:
# Compute correlation of each feature with churn
encoded = pd.get_dummies(df.drop('Churn', axis=1), drop_first=True)
encoded['ChurnFlag'] = df['Churn'].map({'No':0,'Yes':1})
corrs = encoded.corr()['ChurnFlag'].sort_values(ascending=False)

# Display top positive and negative
print("Top positive correlations with churn:")
display(corrs.head(5).round(2))
print("\nTop negative correlations with churn:")
display(corrs.tail(5).round(2))


- **Highest positive correlations** with churn:
  - `InternetService_Fiber optic`: **+0.31**  
  - `PaymentMethod_Electronic check`: **+0.30**  
  - `MonthlyCharges`: **+0.19**  
  - `PaperlessBilling_Yes`: **+0.19**  

- **Highest negative correlations** with churn:
  - `tenure`: **–0.35**  
  - `Contract_Two year`: **–0.30**  
  - `TechSupport_No internet service`: **–0.23**  
  - `OnlineSecurity_No internet service`: **–0.23**  
  - `DeviceProtection_No internet service`: **–0.23**  

> **Note:**  
> - Customers on two‑year contracts and with longer tenures are the most likely to stay.  
> - Fiber‑optic service and electronic-check billing are strong churn risk indicators.  
> - No‑internet‑service categories correlate negatively due to minimal service usage.  
> - Moderate positive correlation of MonthlyCharges suggests bill levels are a lever.  

--- 

### What I noticed

- **People jump ship early.** Over half of the folks who leave do it within their first 10 months. It seems like those first few billing cycles are make‑or‑break.  
- **Big bills seem scary.** Customers who churn pay about \$74 a month on average, versus \$61 for those who stay.  
- **Payment quirks matter.** If someone pays by electronic check, they’re more likely to bail than if they’re on auto‑pay.  
- **Support and extras help.** Not having tech support or extra services (like online security) goes hand‑in‑hand with higher churn—especially for our fiber‑optic users.

---

### Data Quirks & Clean‑Up Notes

- We had to drop just a handful of rows (about 11) because their total charges were missing. That’s less than 0.2% of the data—no biggie.  
- A few customers rack up over \$8,000 in total charges (they’ve been with us forever). We might want to cap or log‑transform those before modeling, but they’re rare.

---

### Where to Go Next

1. **Prep the data for models.** Turn our categories into numbers, scale the big bill amounts, and make sure churn stays balanced in train/test splits.  
2. **Try out a few models.** I’ll start with simple ones—logistic regression, a decision tree or two, maybe a gradient‑boosted model—and see what works best.  
3. **Measure what matters.** We’ll compare accuracy, how many churners each model actually catches, and overall “area under the curve” to pick our winner.  

### Exporting cleaned data

We want to save our cleaned DataFrame (`df`) from the EDA notebook so that milesstone 2 can load it directly. We will write the CSV to the dataset folder.

In [ ]:
# Export cleaned data for modeling
output_path = '../dataset/cleaned_telco_churn.csv'
df.to_csv(output_path, index=False)
print(f"Cleaned data written to {output_path}")